<a href="https://colab.research.google.com/github/Rishabhjuneja/NYC-Taxi/blob/main/NYC_Taxi_Prediction_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data: NYC Taxi (Green Trip data Feb 2016)
# Questions asked:
1. Download the data, load it into python. Report the number of rows and columns that you've loaded.

2. Visualize trip distance by time of day in any way you see fit, any observations?

3. What are the most popular pickup locations on weekends vs weekdays?

4. Build a model to forecast the number of trips by hour for the next 12 hours after Feb 12th 10:00 am. How well did you do?


Created By: Rishabh Juneja 

Date: 9th Aug 2021 

Company: Gale Partners

In [ ]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [ ]:
# Create spark session
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
#Import Libraries
from pyspark.sql import functions as F
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from datetime import datetime
from sklearn.cluster import MiniBatchKMeans, KMeans
import warnings
import itertools
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import time
import folium
from folium import Choropleth ,  Circle, Marker
import datetime as dt
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt 

In [ ]:
# UDF to convert day to day type 
def time_of_day(x):
    if x in range(6,12):
        return 'Morning'
    elif x in range(12,16):
        return 'Afternoon'
    elif x in range(16,22):
        return 'Evening'
    else:
        return 'Late night'

In [ ]:
file_loc='./green_tripdata_2016-02.csv'
taxi_data = spark.read.csv(file_loc, inferSchema=True, header =True)

In [ ]:
taxi_data.show(5)

In [ ]:
# check number of rows & Coloums 

print((taxi_data.count(), len(taxi_data.columns)))

In [ ]:
# Check any missing values 

from pyspark.sql.functions import lit

rows = taxi_data.count()
summary = taxi_data.describe().filter(F.col("summary") == "count")
summary.select(*((F.lit(rows)-F.col(c)).alias(c) for c in taxi_data.columns)).show()

Insights: There is no Missing values in dataset 

In [ ]:
# Descriptive Statistics

desc=taxi_data.describe()

desc.show()
 

Insights: There seems to be outliers in differenet coloums such as  trip distance , fare amount, extra , mat tax should be .5 , trip amount 

In [ ]:
# Check quantile to deepdive check for outliers
pandastaxi_data = taxi_data.toPandas()
pandastaxi_data.quantile(q=0.5, axis=0, numeric_only=True, interpolation='linear')

In [ ]:
# Convert to pandas for plotting graphs and checking outliers
pandas_taxi_data = taxi_data.toPandas()

# Check Trip Distance for outliers

In [ ]:
# Plot trip distance to check outliers
plt.figure(figsize = (8,5))
sns.boxplot("Trip_distance", data = pandas_taxi_data, orient = "v")
plt.tick_params(labelsize = 10)
plt.ylabel("Trip_distance", fontsize = 10)
plt.show()

Study: There are some outliers but we should be looked deep into it

In [ ]:
# We investigate Quartiles (0-100)
tripDistance_quatile = pandas_taxi_data.Trip_distance.quantile(np.round(np.arange(0.00, 1.01, 0.01), 2))
values_q = np.round(np.arange(0.00, 1.01, 0.1), 2)
for i in values_q:
    print("{} miles are in {} percentile ".format( tripDistance_quatile[i],int(i*100)))

90th percentile is 5.98 and 100th is 235.5 so there are outliers and further investigate between 90 to 100

In [ ]:
tripDistance_quatile = pandas_taxi_data.Trip_distance.quantile(np.round(np.arange(0.991, 1.001, 0.001), 4))
values_q = np.round(np.arange(0.991, 1.001, 0.001), 3)
for i in values_q:
    print("{} miles are in {} percentile ".format( tripDistance_quatile[i],int(i*100)))

Insights: since 99.9 percentile is 22.8 and 100 is 235.5, so we will remove anything above 22.8

In [ ]:
# Remove data with outliers of trip distance

data_tripdistance=taxi_data.where("Trip_distance < 22.8")

print((data_tripdistance.count(), len(data_tripdistance.columns)))


# Check fare Amount for outliers

In [ ]:
# Plot Fare_amount to check outliers
plt.figure(figsize = (10,6))
sns.boxplot("Fare_amount", data = pandas_taxi_data, orient = "v")
plt.tick_params(labelsize = 20)
plt.ylabel("Fare_amount", fontsize = 20)
plt.show()

Assumption: fare cant be negative so remove them and some outliers also

In [ ]:
Fare_amount_quatile = pandas_taxi_data.Fare_amount.quantile(np.round(np.arange(0.00, 1.01, 0.01), 2))
values_q = np.round(np.arange(0.00, 1.01, 0.1), 2)
for i in values_q:
    print("{} miles are in {} percentile ".format( Fare_amount_quatile[i],int(i*100)))

In [ ]:
Fare_amount_quatile = pandas_taxi_data.Fare_amount.quantile(np.round(np.arange(0.991, 1.001, 0.001), 3))
values_q = np.round(np.arange(0.991, 1.001, 0.001), 3)
for i in qValues:
    print("{} miles are in {} percentile ".format( Fare_amount_quatile[i],int(i*100)))

Insights: remove anything elow 0 and above 77.5

In [ ]:
# Remove data with outliers of fare amount

data_fareamount=data_tripdistance.where("Fare_amount between 0 and 77.5")

print((data_fareamount.count(), len(data_fareamount.columns)))


# Total Amount Outliers

In [ ]:
total_amount_quatile = pandas_taxi_data.Total_amount.quantile(np.round(np.arange(0.00, 1.01, 0.01), 2))
values_q = np.round(np.arange(0.00, 1.01, 0.1), 2)
for i in values_q:
    print("{} miles are in {} percentile ".format( total_amount_quatile[i],int(i*100)))

In [ ]:
total_amount_quatile = pandas_taxi_data.Total_amount.quantile(np.round(np.arange(0.991, 1.001, 0.001), 3))
values_q = np.round(np.arange(0.991, 1.001, 0.001), 3)
for i in qValues:
    print("{} miles are in {} percentile ".format( total_amount_quatile[i],int(i*100)))

Study: Remove anything below 0 and above 92.2

In [ ]:
# Remove data with outliers of trip distance

data_distance=data_tripdistance.where("Total_amount between 0 and  22.8")

print((data_distance.count(), len(data_distance.columns)))


# Other Outliers 

In [ ]:
# Remove other outliers such as ratecodeid can't be >6 , MTA tax is 0.5, imrovement chargers is 0.3

taxi_data_no_outlier=data_distance.where("RateCodeID < 7").where("MTA_tax  = 0.5").where("improvement_surcharge = 0.3")


print((taxi_data_no_outlier.count(), len(taxi_data_no_outlier.columns)))


Next Steps: Could have looked into other abnormalities like lat and long

# Add new coloums for model & Data Analysis

In [ ]:
# get date and time coloum seprately for pickup and drop
from pyspark.sql import functions as F

split_col = F.split(taxi_data_no_outlier['lpep_pickup_datetime'], ' ')
split_col2 = F.split(taxi_data_no_outlier['Lpep_dropoff_datetime'], ' ')
taxi_data_no_outlier = taxi_data_no_outlier.withColumn('pickup_date', split_col.getItem(0)).withColumn('pickup_time', split_col.getItem(1)).withColumn('drop_date', split_col2.getItem(0)).withColumn('drop_time', split_col2.getItem(1))


taxi_data_no_outlier.show(5)

In [ ]:
# Convert to pandas
taxi_data_no_outlier_pandas = taxi_data_no_outlier.toPandas()

In [ ]:
# Change date time to datetime format
taxi_data_no_outlier_pandas['pickup_datetime']=pd.to_datetime(taxi_data_no_outlier_pandas['lpep_pickup_datetime'])
taxi_data_no_outlier_pandas['dropoff_datetime']=pd.to_datetime(taxi_data_no_outlier_pandas['Lpep_dropoff_datetime'])

# Get day
taxi_data_no_outlier_pandas['pickup_day_number']=taxi_data_no_outlier_pandas['pickup_datetime'].dt.weekday
taxi_data_no_outlier_pandas['dropoff_day_number']=taxi_data_no_outlier_pandas['dropoff_datetime'].dt.weekday

# Get hour 
taxi_data_no_outlier_pandas['pickup_hour']=taxi_data_no_outlier_pandas['pickup_datetime'].dt.hour
taxi_data_no_outlier_pandas['dropoff_hour']=taxi_data_no_outlier_pandas['dropoff_datetime'].dt.hour

# Get day name
taxi_data_no_outlier_pandas['pickup_day']=taxi_data_no_outlier_pandas['pickup_datetime'].dt.day_name()
taxi_data_no_outlier_pandas['dropoff_day']=taxi_data_no_outlier_pandas['dropoff_datetime'].dt.day_name()

# Get time of day
taxi_data_no_outlier_pandas['pickup_timeofday']=taxi_data_no_outlier_pandas['pickup_hour'].apply(time_of_day)
taxi_data_no_outlier_pandas['dropoff_timeofday']=taxi_data_no_outlier_pandas['dropoff_hour'].apply(time_of_day)

In [ ]:
# Display Data 
taxi_data_no_outlier_pandas

# Data Analysis

In [ ]:
# Check pickup and drop of time of day trips

figure,(ax9,ax10)=plt.subplots(ncols=2,figsize=(20,5))

ax9.set_title('Pickup Days')
ax=sns.countplot(x="pickup_hour",data=taxi_data_no_outlier_pandas,ax=ax9)

ax10.set_title('Dropoff Days')
ax=sns.countplot(x="dropoff_hour",data=taxi_data_no_outlier_pandas,ax=ax10)

Insights: 1. Pickup and drop are highest furing evening time i.e. between 4PM and 8 PM

In [ ]:
sns.lineplot(data=taxi_data_no_outlier_pandas, x="pickup_hour", y="Trip_distance")

Insights: trip distance is highest in morning at 5AM fllowed by it sees increase after 4PM till midnight

In [ ]:
# Pickup vs amount
sns.lineplot(data=taxi_data_no_outlier_pandas, x="pickup_hour", y="Total_amount")

Insights: trip amount shows similar behaviour as distance when compared to pikcup hour 

In [ ]:
# amount vs Day
sns.lineplot(data=taxi_data_no_outlier_pandas, x="pickup_day", y="Total_amount")

inisghts: Trip amount is highest on weekend i.e. sat & Sun 

In [ ]:
# amount vs distance
sns.lineplot(data=taxi_data_no_outlier_pandas, x="Total_amount", y="Trip_distance")

Since there seems to correlation amoung variables we will test correlation

In [ ]:
# Plot data based on Lat and Long 
import plotly.express as px

fig = px.scatter_mapbox(taxi_data_no_outlier_pandas, lat="Pickup_latitude", lon="Pickup_longitude", color="pickup_day", zoom=10)
fig.show()

# Check Correlation and Remove for Model

In [ ]:
corr_data=taxi_data_no_outlier_pandas.corr(method ='kendall')

corr_data

There is strong correlation between trip distance with fare amount and total amount so we will remove those variables and other non used variables

# Remove correlated variables 

In [ ]:
# Select Coloums required 
clean_data_pandas=taxi_data_no_outlier_pandas[[	'pickup_datetime'	,'pickup_day_number',	'pickup_hour'	,'pickup_timeofday',	'VendorID',	'Store_and_fwd_flag',	'RateCodeID',		'Passenger_count',	'Trip_distance'	,'Payment_type']]



In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
# change pandas to spark dataframe
cleaned_data = spark.createDataFrame(clean_data_pandas)

# Check data is staionary or not

In [ ]:

from matplotlib import pyplot
import matplotlib.pyplot as plt
clean_data_pandas.plot(x='pickup_datetime')
pyplot.show()

Data is non staionary

# Create final table for Model

In [ ]:
# Change all coloums types of int or float
cleaned_data_strflg=cleaned_data.withColumn("Store_and_fwd_code", F.when(cleaned_data.Store_and_fwd_flag == 'N', F.lit(0)).otherwise(F.lit(1))).drop("Store_and_fwd_flag")

cleaned_data2=cleaned_data_strflg.withColumn("Class",F.when(cleaned_data_strflg.pickup_timeofday == 'Morning', F.lit(1)).when(cleaned_data_strflg.pickup_timeofday == 'Afternoon',F.lit(1)).when(cleaned_data_strflg.pickup_timeofday == 'Evening',F.lit(2)).otherwise(3)).drop("pickup_timeofday").distinct()

cleaned_data2.distinct().count()

In [ ]:
# Add a coloum number of trips for forceasting

trips=cleaned_data2.groupby("pickup_hour","pickup_day_number").agg(F.count(F.lit(1)).alias("trips"))

cleaned_data_final=cleaned_data2.join(trips,(cleaned_data2.pickup_hour==trips.pickup_hour) & (cleaned_data2.pickup_day_number==trips.pickup_day_number)).select(cleaned_data2["*"],trips.trips)

cleaned_data_final.distinct().count()

In [ ]:
# Check max and min pickup time for dividing between test and control 
cleaned_data.agg(F.min("pickup_datetime").alias("min_date"),F.max("pickup_datetime").alias("max_date")).show()

In [ ]:
# Divide test and train (base on date Feb 12th 10:00 am)

train=cleaned_data_final.where("pickup_datetime < '2016-02-12 10:00:00'").drop("pickup_datetime")

test=cleaned_data_final.where("pickup_datetime between '2016-02-12 10:00:00' and '2016-02-12 22:00:00'").drop("pickup_datetime")

In [ ]:
# To pandas
train_pandas = train.toPandas()
test_pandas = test.toPandas()

In [ ]:
# Divide data for model fit
y_train = train_pandas['trips']
x_train = train_pandas.drop(['trips'],axis=1)

y_test = test_pandas['trips']
x_test = test_pandas.drop(['trips'],axis=1)

# Build Model, fit and test

In [ ]:
# get Model details and run it
xgbr = xgb.XGBRegressor(verbosity=0) 
print(xgbr)

In [ ]:
# Model fit
xgbr.fit(x_train, y_train)

In [ ]:
# get Training score
score = xgbr.score(x_train, y_train)  
print("Training score: ", score)

Insights: Training scores are very high shows more accuracy of data

In [ ]:
# Check cross validation score for 2nd check of model

scores = cross_val_score(xgbr, x_train, y_train,cv=10)
print("Mean cross-validation score: %.2f" % scores.mean())


Insights: Cross validation score is 90 % which makes sure model is effecient

In [ ]:
#Get RMSE Score
y_pred = xgbr.predict(x_test)
mse = mean_squared_error(y_test, y_pred)

print("RMSE: %.2f" % (mse**(1/2.0)))


Inisghts: RMSE score is 

In [ ]:
# Plot graph to check accuracy and difference between original and predicted 
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, label="original")
plt.plot(x_ax, y_pred, label="predicted")
plt.title("NYC Taxi trips prediction")
plt.legend()
plt.show()

Insights: graph clearly states that model is predicting values very close to original 